In [2]:
import torch
import dgl
NOTE_PATH = '/nfs3-p1/zty/2021-FanXing/Academic-Networks/Academic_GNN_Module/'
DBLP_PATH = NOTE_PATH + 'dblpv13/'

graph_list, _ = dgl.load_graphs(f'{DBLP_PATH}/dblp.graph')
graph = graph_list[0]
print(graph)

Graph(num_nodes={'author': 3306925, 'country': 54, 'org': 379, 'paper': 5319925, 'topic': 16081},
      num_edges={('author', 'belong', 'org'): 7209864, ('author', 'write', 'paper'): 13024047, ('country', 'contain', 'org'): 432, ('org', 'belong', 'country'): 432, ('org', 'contain', 'author'): 7209864, ('paper', 'belong', 'topic'): 13160952, ('paper', 'written', 'author'): 13024047, ('topic', 'contain', 'paper'): 13160952},
      metagraph=[('author', 'org', 'belong'), ('author', 'paper', 'write'), ('org', 'country', 'belong'), ('org', 'author', 'contain'), ('paper', 'topic', 'belong'), ('paper', 'author', 'written'), ('country', 'org', 'contain'), ('topic', 'paper', 'contain')])


In [5]:
pid, aid = graph.edges(etype='written', order='eid')
print(f'pid:{len(pid)}, {pid}')
print(f'aid:{len(aid)}, {aid}')

pid:13024047, tensor([    175,     273,     273,  ..., 5319924, 5319924, 5319924])
aid:13024047, tensor([1734554,  797305,  761428,  ..., 1828396, 1994351, 1815169])


In [6]:
g1 = dgl.node_type_subgraph(graph, ['author', 'org'])
oid_org, aid_org = g1.edges(etype='contain', order='eid')
print(f'oid_org:{len(oid_org)}, {oid_org}')
print(f'aid_org:{len(aid_org)}, {aid_org}')

oid_org:7209864, tensor([0, 0, 0,  ..., 0, 0, 0])
aid_org:7209864, tensor([      0,       0,       0,  ..., 3306923, 3306924, 3306924])


In [9]:
g2 = dgl.node_type_subgraph(graph, ['country', 'org'])
cid_country, oid_country = g2.edges(etype='contain', order='eid')
print(f'cid_country:{len(cid_country)}, {cid_country[0:10]}')
print(f'oid_country:{len(oid_country)}, {oid_country[:10]}')

cid_country:432, tensor([0, 1, 2, 3, 3, 1, 1, 4, 5, 5])
oid_country:432, tensor([0, 1, 2, 0, 3, 0, 4, 5, 0, 6])


In [10]:
g3 = dgl.node_type_subgraph(graph, ['topic', 'paper'])
tid_topic, pid_topic = g3.edges(etype='contain', order='eid')
print(f'tid_topic:{len(tid_topic)}, {tid_topic}')
print(f'pid_topic:{len(pid_topic)}, {pid_topic}')

tid_topic:13160952, tensor([15592, 11391, 10431,  ...,  7838,  3256,   807])
pid_topic:13160952, tensor([      4,       4,       4,  ..., 5319924, 5319924, 5319924])


In [14]:
year = graph.edges['written'].data['year']
author_org_year = g1.edges['contain'].data['year']
paper_topic_year= g3.edges['contain'].data['year']
print(f'year:{len(year)}, {year}')
print(f'author_org_year:{len(author_org_year)}, {author_org_year}')
print(f'paper_topic_year:{len(paper_topic_year)}, {paper_topic_year}')


year:13024047, tensor([   0.,    0.,    0.,  ..., 9737., 9737., 9737.], dtype=torch.float64)
author_org_year:7209864, tensor([1995., 1997., 1998.,  ..., 2020., 2016., 2020.], dtype=torch.float64)
paper_topic_year:13160952, tensor([   0.,    0.,    0.,  ..., 9737., 9737., 9737.], dtype=torch.float64)


In [15]:
graph_data = {
    ('paper', 'written', 'author'): (pid, aid),
    ('org', 'has', 'author'): (oid_org, aid_org),
    ('country', 'has', 'org'): (cid_country, oid_country),
    ('topic', 'contains', 'paper'): (tid_topic, pid_topic)
}
new_g = dgl.heterograph(graph_data)
new_g.edata['year'] = {
    ('paper', 'written', 'author'): year,
    ('org', 'has', 'author'): author_org_year,
    ('topic', 'contains', 'paper'): paper_topic_year
}
print(new_g)

Graph(num_nodes={'author': 3306925, 'country': 54, 'org': 379, 'paper': 5319925, 'topic': 16081},
      num_edges={('country', 'has', 'org'): 432, ('org', 'has', 'author'): 7209864, ('paper', 'written', 'author'): 13024047, ('topic', 'contains', 'paper'): 13160952},
      metagraph=[('country', 'org', 'has'), ('org', 'author', 'has'), ('paper', 'author', 'written'), ('topic', 'paper', 'contains')])


In [16]:
dgl.save_graphs('../save3/new_g.graph', [new_g])

In [1]:
import numpy as np
pid1 = np.load('../save2/pid1.npy')
pid2 = np.load('../save2/pid2.npy')
year_cite = np.load('../save2/year_cite.npy')
vid_pub = np.load('../save2/vid_pub.npy')
pid_pub = np.load('../save2/pid_pub.npy')
year_pub = np.load('../save2/year_pub.npy')
fid_has = np.load('../save2/fid_has.npy')
vid_has = np.load('../save2/vid_has.npy')
year_has = np.load('../save2/year_has.npy')

In [5]:
print(len(pid1), len(pid2))

28917464 28917464


In [4]:
idx = np.where([pid2>=0])[0]
p1, p2 = pid1[idx], pid2[idx]
print(len(p1), len(p2))

25626821 25626821


In [ ]:
28917464